# Задание 4.2, Панченко С., 674

Загрузим необходимые модули:

In [4]:
import numpy as np
import pandas as pd

from statsmodels.stats.proportion import binom_test
from statsmodels.stats.multitest import multipletests

Для определения того, какая из моделей $a_1,...,a_4$ в среднем даёт лучший результат, воспользуемся двухвыборочным критерием знаков для 
сравнения выборок $X_k$ и $X_m$ (альтернативу выберем односторонней, отдающей предпочтение первой модели):

$$\text{1.Выборки:}$$
$$X_k^{(n)}  = (X_{k1}, ... , X_{kn})$$
$$X_m^{(n)} = (X_{m1}, ... , X_{mn})$$
$$X_{1i} \neq X_{2i} \text{, выборки связанные}$$ 

$$\text{2.Нулевая гипотеза:}$$
$$H_0: \mathbf{P}(X_k > X_m) = \frac{1}{2}$$ 

$$\text{3.Альтернатива:}$$
$$H_1: \mathbf{P}(X_k > X_m) > \frac{1}{2} \text{ (односторонняя)}$$

$$\text{4.Статистика критерия:}$$
$$T(X_k^{(n)}, X_m^{(n)}) = \sum_{i = 1}^n [X_{ki} > X_{mi}]$$

$$\text{5.Распределение статистики в условии истинности }H_0:$$
$$T \text{ ~ } Bin(n, \frac{1}{2})$$

$$\text{6.Правило принятия решения в явном виде: }$$
$$H_0 \text{ отвергается, если p-value } \leq \alpha \text{, где  p-value }= 1 - F_{Bin(n, \frac{1}{2})}(t), t \text{ - значение статистики T} $$

Имеем 4 выборки $X_1,...,X_4$, cоответствующие моделям $a_1,...,a_4$. Отметим, что модели $a_3$ соответствует выборка $X_3$ с наибольшим средним качеством. Применим тогда описанный выше критерий к парам $X_3-X_1$, $X_3-X_2$ и $X_3-X_4$ для определения того, какие из моделей $a_1, a_2, a_4$ хуже $a_3$ - для них нулевая гипотеза окажется отвергнута.

Поскольку мы работаем с тремя гипотезами, то вместо контроля индивидуальных уровней ошибок первого рода будем контролировать FWER на уровне $\alpha = 0.05$, используя для нахождения уровней $\alpha_i$ метод Холма.

Загрузим данные:

In [5]:
data = pd.read_csv('4_2_data.csv')

In [6]:
data

,Номер выборки,a1,a2,a3,a4
0,1,86,50,93,13
1,2,85,74,55,35
2,3,53,92,58,51
3,4,44,41,56,37
4,5,2,18,99,26
5,6,5,68,35,17


Cформируем необходимые для исследования разности выборок и определим, какие из них отвергаются, а какие - нет при контроле FWER.

In [15]:
X = data.values[:, 1:]

In [16]:
D_1 = (X[:, 2] - X[:, 0]) > 0
D_2 = (X[:, 2] - X[:, 1]) > 0
D_3 = (X[:, 2] - X[:, 3]) > 0

p_values = []
for arr in [D_1, D_2, D_3]:
    p_values.append(binom_test(arr.sum(), len(arr), alternative = 'larger'))
    
rejections = multipletests(p_values, method='holm')[0]
print(rejections)

[False False  True]


Итак, применение данного критерия позволило отбросить одну из гипотез, другими словами, модель $a_3$ лучше модели $a_4$ согласно выбранному критерию, в то время как относительно двух других моделей критерий не позволяет принять решения на заданном уровне контроля FWER. Отметим, что в случае модели $a_4$, все соответствующие разности оказываются положительными и отбрасывание соответствующей гипотезы весьма логично - напротив, в случае моделей $a_1, a_2$ некоторые разности отрицательны и критерий не отбрасывает соответствующие гипотезы. Приведенный критерий имеет недостаток - он не учитывает ведичины разностей: модель $a_1$ немного превосходит $a_3$ лишь в одном случае из $6$ и выдает ужасное качество в двух случаях, логично было бы отбросить и её.